In [4]:
%%capture --no-display --no-stderr --no-stdout
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim import Adam
from torch.utils.data import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import os
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
checkpoint = "bigcode/santacoder"
revision = "dedup-alt"
device = "cuda" # for GPU usage or "cpu" for CPU usage

In [6]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint,revision=revision)
model = AutoModelForCausalLM.from_pretrained(checkpoint, revision=revision, trust_remote_code=True).to(device)
tokenizer.pad_token = tokenizer.eos_token

In [7]:
with open("DSL.txt", 'r') as f:
    prompt = f.read()

In [8]:
class CodeDataset(Dataset): # NOTE: this isn't very space-efficient since it has to load in the entire dataset at once
    def __init__(self, data_dir):
        self.data_dir = data_dir

        self.data = []
        for d in os.listdir(self.data_dir):
            code_dir = os.path.join(self.data_dir, d, "code.txt")
            prompts_dir = os.path.join(self.data_dir, d, "prompts.txt")

            with open(code_dir, 'r') as f:
                code = f.read()
            
            with open(prompts_dir, 'r') as f:
                prompts = f.read()
            
            prompts = prompts.split("\n")

            for p in prompts:
                self.data.append((code, prompt + p + "\n\t"))

    def __len__(self):
        return len(os.listdir(self.data_dir))
    def __getitem__(self, idx):
        return self.data[idx]

In [9]:
BATCH_SIZE = 4
dataset = CodeDataset("./train/")
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [10]:
print(dataset[0][1])

'''
def go_to(location : str)
def find(object : str)
def pick_up(object : str)
def put_down(object : str)
def find(object : str)
def ask(person : str, question : str, options: Optional[List[str]])
def say(message : str)
'''
def main():
    # Using the functions defined above, write a script to do the following: Go to the living room to get the TV Remote. Then go to the kitchen to pick up an apple. Give the TV Remote to Joydeep and the apple to Yash. They are both in their respective offices.
	


In [11]:
print(dataset[0][0])

go_to("Living room")
find("TV Remote")
pick_up("TV Remote")
go_to("Kitchen")
find("apple")
pick_up("apple")
go_to("Joydeep's office")
put_down("TV Remote")
go_to("Yash's office")
put_down("apple")


In [12]:
example_tokenized = tokenizer.encode(dataset[0][1], return_tensors="pt").to(device)
example_label = tokenizer.encode(dataset[0][0])

In [13]:
example_output = model.forward(example_tokenized[0])

In [17]:
tokenizer.decode(example_output)

In [6]:
NUM_EPOCHS = 20

model.train()
loss_arr = []

for epoch in range(NUM_EPOCHS):
    running_loss = 0

    for batch in dataloader:
        

GPT2LMHeadCustomModel(
  (transformer): GPT2CustomModel(
    (wte): Embedding(49280, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2CustomBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2MQAttention(
          (q_attn): Conv1D()
          (kv_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): FastGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2CustomBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2MQAttention(
          (q_attn): Conv1D()
          (kv_attn): Conv1D()
          (c_proj): C